In [1]:
import csv
import os
import numpy as np
import pandas as pd
import gc
from datetime import timedelta, date
import uuid
import traceback

import config
import queries as q
import tools

pd.options.mode.chained_assignment = None
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [2]:
def upd_main_with_t30():
    select_qry = """
    select distinct header_id from trafc.electronic_count_data_partitioned
    where light_motor_vehicles is null
    ;
    """

    df = pd.read_sql(select_qry, config.ENGINE)
    df.dropna(subset=["header_id"], inplace=True)
    list = df.header_id.astype(str).to_list()
    cs1 = pd.read_sql_query(q.SELECT_CLASSIFICAITON_SCHEME_1, config.ENGINE)
    cs5 = pd.read_sql_query(q.SELECT_CLASSIFICAITON_SCHEME_5, config.ENGINE)
    cs8 = pd.read_sql_query(q.SELECT_CLASSIFICAITON_SCHEME_8, config.ENGINE)
    for header_id in list:
        df30 = pd.read_sql(f"""
            select * from trafc.electronic_count_data_type_30 where header_id = '{header_id}';
            """
            , config.ENGINE)
        

        df30 = df30.groupby(['site_id', 'start_datetime', 'lane_number', 'classification_scheme', 'class_number'])['number_of_vehicles'].sum()

        for index, row in df30.iteritems():
            site_id = index[0]
            start_datetime = index[1]
            lane_number = index[2]
            classification_scheme = index[3]
            class_number = index[4]
            number_of_vehicles =  row
            if classification_scheme == 1:
                update_qry = f"""
                    update trafc.electronic_count_data_partitioned set
                    {cs1['vehicle'].at[class_number].lower().replace(to_replace=[" ","-"],value="_").replace(to_replace=[",","(",")"],value="")} = {number_of_vehicles}
                    where site_id = '{site_id}'
                    and start_datetime = '{start_datetime}'
                    and lane_number = {lane_number}
                    and {cs1['vehicle'].at[class_number].lower().replace(to_replace=[" ","-"],value="_").replace(to_replace=[",","(",")"],value="")} is null;
                """
            elif classification_scheme == 5:
                update_qry = f"""
                    update trafc.electronic_count_data_partitioned set 
                    {cs5['vehicle'].at[class_number].lower().replace(to_replace=[" ","-"],value="_").replace(to_replace=[",","(",")"],value="")} = {number_of_vehicles}
                    where site_id = '{site_id}' 
                    and start_datetime = '{start_datetime}'
                    and lane_number = {lane_number}
                    and {cs5['vehicle'].at[class_number].lower().replace(to_replace=[" ","-"],value="_").replace(to_replace=[",","(",")"],value="")} is null;            
                """
            elif classification_scheme == 8:
                update_qry = f"""
                    update trafc.electronic_count_data_partitioned set 
                    {cs8['vehicle'].at[class_number].lower().replace(to_replace=[" ","-"],value="_").replace(to_replace=[",","(",")"],value="")} = {number_of_vehicles}
                    where site_id = '{site_id}' 
                    and start_datetime = '{start_datetime}'
                    and lane_number = {lane_number}
                    and {cs8['vehicle'].at[class_number].lower().replace(to_replace=[" ","-"],value="_").replace(to_replace=[",","(",")"],value="")} is null;            
                """
            else:
                pass
            try:
                with config.CONN as conn:
                    cur = conn.cursor()
                    cur.execute(update_qry)
                    conn.commit()
            except Exception as e:
                print(e)
                print(update_qry)
                traceback.print_exc()
                conn.rollback()
                conn.close()
                gc.collect()
                break
    print("Done")

In [3]:
upd_main_with_t30()